In [2]:

import os
import pandas as pd


home = os.environ["HOME"]
os.chdir(f"{home}/tiniMLVM")
sensor_data_folder = "smart_city/spire_2023.csv"
df = pd.read_csv(sensor_data_folder, sep=";")

df.head()

,data,codice_spira,00_00_01_00,01_00_02_00,02_00_03_00,03_00_04_00,04_00_05_00,05_00_06_00,06_00_07_00,07_00_08_00,...,ordinanza,stato,codimpsem,direzione,angolo,longitudine,latitudine,geopoint,chiave,num_giorno_settimana
0,2023-04-01,0.127 1.5 2 1,34,31,14,7,9,12,22,48,...,4000/343434,A,250,S,163.0,11.343595,44.501688,"44.5016875497437, 11.3435953508993",16.0,Sabato
1,2023-04-01,0.127 1.5 5 1,38,29,20,6,13,12,18,21,...,4000/343434,A,250,N,343.0,11.343408,44.501173,"44.5011734938257, 11.3434083495971",17.0,Sabato
2,2023-04-01,0.127 1.13 6 1,0,0,0,0,0,0,0,0,...,4000/343434,A,323,S,198.0,11.346420,44.491648,"44.4916483847646, 11.3464200565732",3.0,Sabato
3,2023-04-01,0.127 1.15 6 1,68,50,38,13,12,18,32,43,...,4000/343434,A,324,S,168.0,11.343443,44.490027,"44.4900271601578, 11.3434427397635",7.0,Sabato
4,2023-04-01,0.127 1.82 2 2,86,55,37,30,27,23,51,119,...,4000/343434,A,85,SE,217.0,11.349800,44.490458,"44.4904584072207, 11.34979980231",20.0,Sabato


In [5]:
print(df.columns)

Index(['data', 'codice_spira', '00_00_01_00', '01_00_02_00', '02_00_03_00',
       '03_00_04_00', '04_00_05_00', '05_00_06_00', '06_00_07_00',
       '07_00_08_00', '08_00_09_00', '09_00_10_00', '10_00_11_00',
       '11_00_12_00', '12_00_13_00', '13_00_14_00', '14_00_15_00',
       '15_00_16_00', '16_00_17_00', '17_00_18_00', '18_00_19_00',
       '19_00_20_00', '20_00_21_00', '21_00_22_00', '22_00_23_00',
       '23_00_24_00', 'id_uni', 'livello', 'tipologia', 'codice',
       'codice_arco', 'codice_via', 'nome_via', 'nodo_da', 'nodo_a',
       'ordinanza', 'stato', 'codimpsem', 'direzione', 'angolo', 'longitudine',
       'latitudine', 'geopoint', 'chiave', 'num_giorno_settimana'],
      dtype='object')


In [ ]:
#get a spire key and save all his values at rows in a file

def get_spire_key_values(key):
    spire_key = df[df["chiave"] == key]

    position = spire_key["geopoint"].values[0]

    position = [float(i) for i in position.split(",")]

    #drop the non 2 to 25 columns 
    columns_keep = [df.columns[i] for i in range(2, 26)]
    spire_key = spire_key[columns_keep]
    #stack the columns in a single column
    spire_key = spire_key.stack()
    #save the values in a file
    spire_key.to_csv(f"sensor_manager/sensor_data/spire_{key}_{position}.csv", index=False)

get_spire_key_values(5)

In [ ]:
import numpy as np
import math

#get geopoints

list_geopoints = []

for file in os.listdir(f"{home}/tiniMLVM/sensor_manager/sensor_data"):
    if "spire" in file:
        position = file.split("_")[-1].replace(".csv", "")
        lat1, lon1 = float(position.split(",")[0][1:]), float(position.split(",")[1][:-1])
        list_geopoints.append((lat1, lon1))


def get_distance(lat1, lon1, lat2, lon2):
    r = 6371  # Radius of the earth in km

    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = r * c  # Distance in km
    return distance

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the Earth (specified in decimal degrees)
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1  # Difference in latitude
    dlon = lon2 - lon1  # Difference in longitude
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    # Radius of Earth in kilometers (can be changed to 3958.8 for miles)
    r = 6371  # Radius of Earth in km
    distance = r * c  # Distance in km
    return distance


for i in range(len(list_geopoints)):
    for j in range(i + 1, len(list_geopoints)):
        distance = get_distance(list_geopoints[i][0], list_geopoints[i][1], list_geopoints[j][0], list_geopoints[j][1])
        print(f"Distance between {list_geopoints[i]} and {list_geopoints[j]} is {distance} km")

Distance between (44.4901162203284, 11.3398356513878) and (44.4985349106485, 11.3541657967424) is 1.4724961270775772 km
Distance between (44.4901162203284, 11.3398356513878) and (44.4985349106485, 11.3541657967424) is 1.4724961270775772 km
